In [2]:
#import necessary libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
#load webpage https://www.transfermarkt.co.uk
# headers will be used to trick the website that we open it like a browser, not like a scrapping tool. 
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
page = 'https://www.transfermarkt.co.uk/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&altersklasse=alle&ausrichtung=alle&jahrgang=0&kontinent_id=0&land_id=0&page=1&plus=1&spielerposition_id=alle'
try:
    content = requests.get(page, headers=headers)
except:
    print('Source 1 unavailable. Check the internet connection')
soup = BeautifulSoup(content.content, 'html.parser')
#identify columns from content page
player_table = soup.find("table", attrs={"class": "items"})
columns = []
for th in player_table.find_all('th'):
    title = th.text
    if title !='':
        columns.append(title)
    else:
        columns.append(th.find('div').get('title')) # in the website not all headers have values, it uses icons, therefore we get title value  
# below when we scrap the data we will get some unnecessary items
# we create some extra columns, then we will drop them 
columns.insert(1, "delete1")
columns.insert(2, "delete2")
columns.insert(4, "position")
columns2 = ["delete3","delete4", "delete5"]
columns.extend(columns2)
# create DataFrame with necessary columns 
data_tm = pd.DataFrame(columns = columns)

# Because of necessary data are divided into 20 webpages, we will use for loop  
for i in range(1,21):
    page = 'https://www.transfermarkt.co.uk/spieler-statistik/wertvollstespieler/marktwertetop?ajax=yw1&altersklasse=alle&ausrichtung=alle&jahrgang=0&kontinent_id=0&land_id=0&page=' + str(i) + '&plus=1&spielerposition_id=alle'
    try:
        content = requests.get(page, headers=headers)
    except:
        print('Source 1 unavailable. Check the internet connection')
    soup = BeautifulSoup(content.content, 'html.parser')
    #scrap data from content page
    listo = []
    player_table = soup.find("table", attrs={"class": "items"})
    for tr in player_table.find_all('tr'):
        data = tr.find_all('td')
        for tr in data:
            item = tr.text
            if  item !='':
                listo.append(item)
            else:
                listo.append(tr.find('img').get('alt')) # where images are used instead of item, we get information from images.
    # insert data into DataFrame
    for i in range(0, len(listo), len(columns)):
        kalisto = listo[i:i + len(columns)]            
        data_tm_length = len(data_tm)
        data_tm.loc[data_tm_length] = kalisto

In [4]:
# make copy of data_frame
df_source1 = data_tm.copy()

In [5]:
# rename a column
df_source1.rename(columns={'Player': 'name'}, inplace = True)

In [6]:
# leave only necessary columns
df_source1 = df_source1[['name', 'position', 'Age', 'Nat.', 'Market value', 'club', 'Goals', 'Assists']] 

In [7]:
# checking our DataFrame
df_source1

,name,position,Age,Nat.,Market value,club,Goals,Assists
0,Kylian Mbappé,Centre-Forward,22,France,£144.00m,Paris Saint-Germain,9,12
1,Erling Haaland,Centre-Forward,21,Norway,£135.00m,Borussia Dortmund,14,4
2,Harry Kane,Centre-Forward,28,England,£108.00m,Tottenham Hotspur,8,2
3,Jack Grealish,Left Winger,26,England,£90.00m,Manchester City,2,3
4,Mohamed Salah,Right Winger,29,Egypt,£90.00m,Liverpool FC,17,7
...,...,...,...,...,...,...,...,...
495,Giorgian de Arrascaeta,Attacking Midfield,27,Uruguay,£16.20m,Clube de Regatas do Flamengo,0,0
496,Ayoze Pérez,Second Striker,28,Spain,£16.20m,Leicester City,1,3
497,Alex Meret,Goalkeeper,24,Italy,£16.20m,SSC Napoli,0,0
498,Duje Caleta-Car,Centre-Back,25,Croatia,£16.20m,Olympique Marseille,0,0


In [8]:
# export our data into csv file.
df_source1.to_csv('../data/df_source1.csv', index=False)